In [ ]:
import requests, re, time
import torch, torchvision
from torch import nn, optim
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt

In [ ]:
#load dataset into directory from drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
xform = transforms.Compose([transforms.Resize((224,224)), transforms.ToTensor()])
dataset_full = datasets.ImageFolder('/content/drive/MyDrive/dataset', transform=xform)

In [ ]:
dataset_full

Dataset ImageFolder
    Number of datapoints: 2259
    Root location: /content/drive/MyDrive/dataset
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear)
               ToTensor()
           )

In [ ]:
dataset_full.class_to_idx

{'angry': 0, 'happy': 1, 'neutral': 2, 'sad': 3}

In [ ]:
dataset_full.classes

['angry', 'happy', 'neutral', 'sad']

According to the second link cited in the proposal, a grayscale transofrm on top of resizing has given the best results when comparing sentiments on faces


In [ ]:
#any possible data augmentation here:






In [ ]:
n_all = len(dataset_full)
n_train = int(0.8 * n_all)
n_test = n_all - n_train
rng = torch.Generator().manual_seed(1549)
dataset_train, dataset_test = torch.utils.data.random_split(dataset_full, [n_train, n_test], rng)
loader_train = torch.utils.data.DataLoader(dataset_train, batch_size = 4, shuffle=True)
loader_test = torch.utils.data.DataLoader(dataset_test, batch_size = 4, shuffle=True) #prepare dataset by splitting, same as usual

In [ ]:
model = models.resnet18(pretrained=True)

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth


In [ ]:
torch.cuda.device_count()

1

In [ ]:
device = torch.device('cuda:0')
model = model.to(device)

as done in A2, need to modify layers to attain better accuracies later on. could experiment on raw resnet first and modify fc layers later on.

In [ ]:
criterion = nn.CrossEntropyLoss()

def run_test(model):
    nsamples_test = len(dataset_test)
    loss, correct = 0, 0
    model.eval()
    with torch.no_grad():
        for samples, labels in loader_test:
            samples = samples.to(device)
            labels = labels.to(device)
            outs = model(samples)
            loss += criterion(outs, labels)
            _, preds = torch.max(outs.detach(), 1)
            correct_mask = preds == labels
            correct += correct_mask.sum(0).item()
    return loss / nsamples_test, correct / nsamples_test

In [ ]:
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

In [ ]:
def run_train(model, opt, sched):
    nsamples_train = len(dataset_train)
    loss_sofar, correct_sofar = 0, 0
    model.train()
    with torch.enable_grad():
        for samples, labels in loader_train:
            samples = samples.to(device)
            labels = labels.to(device)
            opt.zero_grad()
            outs = model(samples)
            _, preds = torch.max(outs.detach(), 1)
            loss = criterion(outs, labels)
            loss.backward()
            opt.step()
            loss_sofar += loss.item() * samples.size(0)
            correct_sofar += torch.sum(preds == labels.detach())
    sched.step()
    return loss_sofar / nsamples_train, correct_sofar / nsamples_train

In [ ]:
def run_all(model, optimizer, scheduler, n_epochs):
    for epoch in range(n_epochs):
        loss_train, acc_train = run_train(model, optimizer, scheduler)
        loss_test, acc_test = run_test(model)
        print(f"epoch {epoch}: train loss {loss_train:.4f} acc {acc_train:.4f}, test loss {loss_test:.4f} acc {acc_test:.4f}")

In [ ]:
run_test(model)

(tensor(2.3835, device='cuda:0'), 0.0)

In [ ]:
run_all(model, optimizer, scheduler, 10)

/usr/local/lib/python3.7/dist-packages/PIL/Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


epoch 0: train loss 1.5644 acc 0.4792, test loss 0.2532 acc 0.5929
epoch 1: train loss 0.9198 acc 0.6624, test loss 0.2972 acc 0.5907
epoch 2: train loss 0.7019 acc 0.7476, test loss 0.1589 acc 0.7699
epoch 3: train loss 0.5388 acc 0.8113, test loss 0.1697 acc 0.7876
epoch 4: train loss 0.3919 acc 0.8605, test loss 0.1694 acc 0.7611
epoch 5: train loss 0.2698 acc 0.9081, test loss 0.1184 acc 0.8429
epoch 6: train loss 0.1605 acc 0.9507, test loss 0.1168 acc 0.8407
epoch 7: train loss 0.1428 acc 0.9585, test loss 0.1144 acc 0.8518
epoch 8: train loss 0.1558 acc 0.9568, test loss 0.1192 acc 0.8540
epoch 9: train loss 0.1453 acc 0.9640, test loss 0.1046 acc 0.8562


In [ ]:
# https://ieeexplore.ieee.org/document/8990979, maximum expected acc is 70% using resnet18 and kaggle dataset